##Digit Classification

In [1]:
####Libraries

#Standard Library
import random

#Third-party Libraries
import numpy as np

- np.random.randn will generate Gaussian distribution with mean 0, sd 
- Random initilizaiton gives our schotastic gradient descent good starting place

In [24]:
class Network():
    
    def __init__(self, sizes): #sizes is a list of the number of neurons in the respective layers
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #shape of (y,1) because input nodes have no bias
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #[:-1] is all but last elem
    
    def feedforward(self, a):
        """
        Return output of the network if 'a' is input
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid_vec(np.dot(w, a) + b) # our vector of activations
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """
        Training_data: list of tuples (x,y) -->training inputs and desired output
        eta: learning rate
        epochs: number of epochs to train for
        mini_batch_size: size of mini batch to use when sampling
        *If test_data we evaluate network against test data after eah eopoch and print parial progress 
        NOTE * will cause serious performance hit
        """
        if test_data: 
            n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data) #shuffle training data
            mini_batches = [ #partition into minibatches of mini_batch_size
                training_data[k: k + mini_batch_size] for k in xrange(0, n, mini_batch_size)
            ]
            for mini_batch in mini_batches: #apply single SGD for each mini_batch
                self.update_mini_batch(mini_batch, eta) #update network weights, biases according to single SGD iteration
            if test_data:
                print('Epoch {0}: {1} / {2}'.format(j, self.evaluate(test_data), n_test))
            else:
                print('Epoch {0} complete'.format(j))
                
    def update_mini_batch(self, mini_batch, eta):
        """
        Mini_batch: list of tuples of size(x, y)
        eta: learning rate
        Update the network's weights and biases by applying gradient descent
        using backpropogation to a single mini-batch
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases] #nabla is gradient symbol 
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            #self.backprop is fast way of computing the gradients for every training exp in mini_batch
            #then update our weights and biases appropriately
            #self.backprop should return the appropriate gradient for the cost associated to the training example x
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] 
        #new weights, biases after a training epoch
        self.weights = [w-(eta/len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)] #eq 20
        self.biases =  [b-(eta/len(mini_batch)) * nb for b, nb in zip(self.biases,  nabla_b)] #eq 21
        
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid_vec(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime_vec(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            spv = sigmoid_prime_vec(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * spv
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        """
        Return number of test inputs for which the network outputs the correct results
        The nets output is assumed to be the index of whichever neuron in last layer has highest activtaion
        """
        test_results = [(np.argmax(self.feedforward(x)), y) for (x,y) in test_data]
        return sum(int(x==y) for (x,y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        """
        Return vector of partial derivatives \partial C_x / \partial a for the output activations
        """
        return output_activations-y
    

In [14]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

#vectorized form of sigmoid function
# takes a nested sequence of objects or numpy arrays as inputs and returns a numpy array as output
sigmoid_vec = np.vectorize(sigmoid)

def sigmoid_prime(z):
    """Derivative of sigmoid func"""
    return sigmoid(z)*(1-sigmoid(z))

sigmoid_prime_vec = np.vectorize(sigmoid_prime)

In [14]:
net = Network([2,3,1])

[2, 3]


In [15]:
#Biases, Weights stored as numpy matrices
net.weights

[array([[-0.23856424,  0.07804673],
        [ 1.5756189 , -0.17127159],
        [ 0.22516764, -0.68210178]]),
 array([[-0.20511055,  1.75351302,  0.0821564 ]])]

###Load Up Our Data

In [2]:
import mnist_loader

In [3]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [25]:
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8981 / 10000
Epoch 1: 9247 / 10000
Epoch 2: 9246 / 10000
Epoch 3: 9376 / 10000
Epoch 4: 9397 / 10000
Epoch 5: 9419 / 10000
Epoch 6: 9414 / 10000
Epoch 7: 9431 / 10000
Epoch 8: 9431 / 10000
Epoch 9: 9431 / 10000
Epoch 10: 9458 / 10000
Epoch 11: 9481 / 10000
Epoch 12: 9465 / 10000
Epoch 13: 9464 / 10000
Epoch 14: 9480 / 10000
Epoch 15: 9481 / 10000
Epoch 16: 9525 / 10000
Epoch 17: 9504 / 10000
Epoch 18: 9520 / 10000
Epoch 19: 9510 / 10000
Epoch 20: 9507 / 10000
Epoch 21: 9526 / 10000
Epoch 22: 9501 / 10000
Epoch 23: 9527 / 10000
Epoch 24: 9536 / 10000
Epoch 25: 9523 / 10000
Epoch 26: 9513 / 10000
Epoch 27: 9525 / 10000
Epoch 28: 9529 / 10000
Epoch 29: 9543 / 10000
